In [3]:
#'''import libraries and tables and connect to the .gdb
import arcpy
import pandas as pd
arcpy.env.workspace = 'C:/Users/e131971/Desktop/Archive/ArcPro/ArcPro.gdb'
fields = pd.read_excel('C:/Users/e131971/Desktop/Archive/2025/5/SNBIBridgeFields/SNBI_Fields.xlsx',sheet_name='Bridge')
try:
    arcpy.management.CreateFeatureclass('C:/Users/e131971/Desktop/Archive/ArcPro/ArcPro.gdb', 'feature', geometry_type= 'POINT')
except:
    pass
feature = 'C:/Users/e131971/Desktop/Archive/ArcPro/ArcPro.gdb/feature'
#'''

#'''if this is a fresh import of SNBI_Fields.xlsx's Bridge sheet, remove the duplicate record
fields = fields.drop_duplicates()
# and filter to get the B codes
fields = fields[fields['Name'].str[0] == 'B']
#'''

#'''separate the fields
name = fields['Name']
descName = name.copy()
#desc = fields['Description']
#cType = fields['Control Type']
dataType = fields['Data Type'] #where Name = 'B.PS.01: Load Posting Status' is missing this info in the original spreadsheet
maxLen = fields['Max Length']
decimals = fields['Number of Decimal Places']
#mappedForms = fields['Mapped Forms']
#comments = fields['Comments']
#'''

#'''formatting
replacer = str.maketrans('. -,','____',':()')
pd.options.mode.chained_assignment = None  # default='warn'
for i in name.index:
    name[i] = name[i].translate(replacer)
dataType = (dataType.replace('String','TEXT').replace('Integer','LONG').replace('Decimal','FLOAT').replace('Date','DATE'))
#'''

#'''import schema
for i in dataType.index:
    if dataType[i] == 'TEXT':
        arcpy.management.AddField(feature, name[i], dataType[i], field_length=maxLen[i], field_alias=descName[i])
    elif dataType[i] == 'DATE':
        arcpy.management.AddField(feature, name[i], dataType[i], field_alias=descName[i])
    elif dataType[i] == 'LONG':
        arcpy.management.AddField(feature, name[i], dataType[i], field_precision=maxLen[i], field_alias=descName[i])
    elif dataType[i] == 'FLOAT':
        if isinstance(dataType[i], int):
            arcpy.management.AddField(feature, name[i], dataType[i], field_precision=maxLen[i], field_scale=decimals[i], field_alias=descName[i])
        else:
            arcpy.management.AddField(feature, name[i], dataType[i], field_precision=maxLen[i], field_alias=descName[i])
    else:
        print(str(i) + ' Data type not recognised for ' + descName[i] + 'I believe B.PS.01: Load Posting Status is a float')
        arcpy.management.AddField(feature, name[i], 'FLOAT', field_precision=maxLen[i], field_alias=descName[i])
    #print(i)
#'''

#'''Take a look
look = pd.DataFrame(arcpy.da.TableToNumPyArray(feature, "*"))
#'''

#'''delete feature to try again
#arcpy.Delete_management(feature)
#'''

In [4]:
#'''import libraries and tables
import arcpy
import pandas as pd
arcpy.env.workspace = 'C:/Users/e131971/Desktop/Archive/ArcPro/ArcPro.gdb'
domains = pd.read_excel('C:/Users/e131971/Desktop/Archive/2025/5/SNBIBridgeFields/SNBI_Fields.xlsx',sheet_name='Bridge Dropdown')
fields = pd.read_excel('C:/Users/e131971/Desktop/Archive/2025/5/SNBIBridgeFields/SNBI_Fields.xlsx',sheet_name='Bridge')
#'''

#'''if this is a fresh import of SNBI_Fields.xlsx's Bridge sheet, remove the duplicate record
fields = fields.drop_duplicates()
# and filter to get the B codes
fields = fields[fields['Name'].str[0] == 'B']
fields['Data Type'] = fields['Data Type'].replace('String','TEXT').replace('Integer','LONG').replace('Decimal','FLOAT').replace('Date','DATE')
#'''

#'''set variables
gdb = 'C:/Users/e131971/Desktop/Archive/ArcPro/ArcPro.gdb'
dName = domains['Name']
dNameList = [] #domain name, can have special characters
fNameList = [] #feature name, must match names in AttributeLoader cannot have special characters other than _
fName = fields['Name']
dataType = fields['Data Type']
indexList = []
dataTypeList = []
optName = domains['Option Name']    #Name/Desc
optValue = domains['Option Value']  #Code
feature = 'C:/Users/e131971/Desktop/Archive/ArcPro/ArcPro.gdb/feature'
replacer = str.maketrans('. -,','____',':()') # characters in, out, delete
#'''

#'''formatting and populating
pd.options.mode.chained_assignment = None  # default='warn'
for i in dName.index:
    if isinstance(dName[i], str): #if record is str, places it in a list
        dNameList.append(dName[i])
        fNameList.append(dName[i])
    else:
        dName[i] = dName[i-1] #if record is blank, copies above record
fNameList = pd.Series(fNameList)
for i in fNameList.index:
    fNameList[i] = fNameList[i].translate(replacer) #formats fNameList

for i in optName.index:
    if not isinstance(optName[i], str): #get rid of rows with empty data
        dName.drop([i], inplace=True)
        optName.drop([i], inplace=True)
        optValue.drop([i], inplace=True)
matcher = dName.copy()

for i in dNameList:
    indexList.append(fName[fName == i].index[0])
for i in indexList:
    dataTypeList.append(dataType[i])

matcher = pd.merge(dName, fields, how='left', on='Name')
dName = matcher['Name']
dDataType = matcher['Data Type']
#'''

# #'''create the domains
# for i in range(len(dNameList)):
#     arcpy.management.CreateDomain(gdb, dNameList[i], field_type=dataTypeList[i])
# #'''

#'''some more formatting
#dNameList = dNameList.values.tolist()
fNameList = fNameList.values.tolist()
dName.reset_index(drop=True, inplace=True)
optValue.reset_index(drop=True, inplace=True)
optName.reset_index(drop=True, inplace=True)
#'''

# #'''populate the domains (this takes like 10 min)
# for i in dName.index:
#     if dDataType[i] == 'LONG':
#         arcpy.management.AddCodedValueToDomain(gdb, dName[i], int(optValue[i]), optName[i])
#     elif dDataType[i] == 'TEXT':
#         arcpy.management.AddCodedValueToDomain(gdb, dName[i], str(optValue[i]), optName[i])
#     #print(i)
# #'''

#'''assign the domains
for i in range(len(fNameList)):
    try:
        arcpy.management.AssignDomainToField(feature, fNameList[i], dNameList[i])
    except arcpy.ExecuteError:
        print("Error Assigning Domain to Field ", fNameList[i])
        '''
        Error Assigning Domain to Field  B_SP_06_Span_Type
        Error Assigning Domain to Field  B_SB_04_Substructure_Type
        Error Assigning Domain to Field  B_AP_02_Overtopping_Likelihood
        Error Assigning Domain to Field  B_AP_03_Scour_Vulnerability
        Error Assigning Domain to Field  B_AP_04_Scour_Plan_of_Action
        '''
#'''

Error Assigning Domain to Field  0 B_L_10_Border_Bridge_Designated_Lead_State
Error Assigning Domain to Field  15 B_SP_06_Span_Type
Error Assigning Domain to Field  24 B_SB_04_Substructure_Type
Error Assigning Domain to Field  111 B_AP_02_Overtopping_Likelihood
Error Assigning Domain to Field  112 B_AP_03_Scour_Vulnerability
Error Assigning Domain to Field  113 B_AP_04_Scour_Plan_of_Action
